In [55]:
import io, pandas as pd, json, os
from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
from joblib import Parallel, delayed
from tqdm import tqdm
from itertools import product
from itertools import permutations
from itertools import combinations
from pyEDM import *
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Ridge
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.base import BaseEstimator, TransformerMixin
import time
import os
from copy import deepcopy
import math
import random
from sklearn.metrics import root_mean_squared_error
from scipy.stats import ttest_ind
import pickle
import ast
import json



from IPython.display import display, HTML
display(HTML('<style>.container { width:90% !important; }</style>'))

import warnings
warnings.filterwarnings("ignore", 
    message="A worker stopped while some jobs were given to the executor.",
    module="joblib.externals.loky.process_executor")

In [117]:
import io, pandas as pd
from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload

FILE_ID = "1bHrmzw5W5cz8DE-ggEwA3WSEDuszG72UUvKQRBBnLPA"     # your sheet
SERVICE_JSON = "/Users/athulithparaselli/Documents/GitHub/HAB-Prediction-Research/Code/APIKey/sacred-truck-461716-t5-2a6499bf0869.json"

# --- auth ---
creds = service_account.Credentials.from_service_account_file(
    SERVICE_JSON,
    scopes=["https://www.googleapis.com/auth/drive.readonly"],
)
drive = build("drive", "v3", credentials=creds)

# --- export Google Sheet -> CSV ---
request = drive.files().export_media(
    fileId=FILE_ID,
    mimeType="text/csv",
    # **required for Shared Drives** :contentReference[oaicite:0]{index=0}
)

fh = io.BytesIO()
downloader = MediaIoBaseDownload(fh, request)
done = False
while not done:
    status, done = downloader.next_chunk()          # progress in chunks

fh.seek(0)                     # rewind the buffer
df = pd.read_csv(fh)           # straight into pandas
print(df.head())


   SampleID Location Code Date Collected (YYYY_MM_DD)  \
0  20080630           SIO                     6/30/08   
1  20080707           SIO                      7/7/08   
2  20080714           SIO                     7/14/08   
3  20080721           SIO                     7/21/08   
4  20080728           SIO                     7/28/08   

  Time Collected PT (hh:mm:ss)  Temp (deg C)  Chl Volume Filtered (mL)  \
0                      8:00:00          19.8                       100   
1                      8:00:00          17.3                       100   
2                      7:54:00          21.7                       100   
3                      7:50:00          20.2                       100   
4                      9:44:00          22.4                       100   

   Chl1 (mg/m3)  Chl2 (mg/m3)  Avg Chloro (mg/m3)  Phaeo1 (mg/m3)  ...  \
0          1.81          1.83                1.82            0.59  ...   
1          2.68          2.73                2.71            1.2

In [115]:
df['Lingulodinium polyedra (cells/L)']

0        NaN
1        NaN
2        NaN
3        NaN
4        NaN
       ...  
878      0.0
879      0.0
880      0.0
881      0.0
882    500.0
Name: Lingulodinium polyedra (cells/L), Length: 883, dtype: float64

In [110]:
data_path = "Data/d1carter_data_w_gaps.csv"
model_path = "/Users/athulithparaselli/Documents/GitHub/HAB-Prediction-Research/Code/Results/800_sample_bioluminescent_driver_carter_models.csv"
def str_to_list(s):
    s = s.replace('nan', 'null')  # Replace 'nan' with 'null' for JSON compatibility
    lst = json.loads(s)  # Convert string to list
    lst = [np.nan if x is None else x for x in lst]  # Replace None with np.nan
    return lst

def process_parameters(path):

    parameters = pd.read_csv(path) 
    parameters['pred'] = parameters['pred'].apply(str_to_list)
    parameters['columns'] = parameters['columns'].apply(ast.literal_eval)
    parameters.sort_values(by='rho',ascending=False)
    return parameters

def clean_data(data, path=True):
    if path:
        paper_data = pd.read_csv(data)
    else:
        paper_data=data
    #paper_data = adjust_live_data(df)
    paper_data = paper_data.set_index('time')
    corr_date = paper_data['DATE']
    paper_data['Time'] = paper_data.index.astype(int)
    paper_data['Avg_Chloro'] #= paper_data['Avg_Chloro'].apply(np.log1p) #LOG AMPUTATION
    #IMPUTE HAB DATA
    #Build basic linear regression model as sanity check
    # Custom impute missing values with the average of the value in front and behind of it 
    class ForwardBackwardImputer(BaseEstimator, TransformerMixin):
        def __init__(self):
            pass

        def fit(self, X, y=None):
            return self

        def transform(self, X):
            X_filled_forward = X.fillna(method='ffill').fillna(method='bfill')
            X_filled_backward = X.fillna(method='bfill').fillna(method='ffill')

            return (X_filled_forward + X_filled_backward) / 2


    Imputer = ForwardBackwardImputer()
    paper_data = paper_data.apply(pd.to_numeric, errors='coerce')
    Imputer.fit(paper_data)
    paper_data = Imputer.transform(paper_data)#COMMENT OUT IF DONT WANT MEAN MPUTE
    paper_data['DATE'] = corr_date
    return paper_data

In [104]:

data = clean_data(data_path)
parameters = process_parameters(model_path)
target = 'Lingulodinium_polyedra'#'Avg_Chloro'#'Lingulodinium_polyedra'#


/var/folders/5y/74w_zv855875xvjtmhp4g1xw0000gn/T/ipykernel_75917/260246522.py:35: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  X_filled_forward = X.fillna(method='ffill').fillna(method='bfill')
/var/folders/5y/74w_zv855875xvjtmhp4g1xw0000gn/T/ipykernel_75917/260246522.py:36: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  X_filled_backward = X.fillna(method='bfill').fillna(method='ffill')


In [105]:
data['DATE']


time
0      2008-06-30
1      2008-07-07
2      2008-07-14
3      2008-07-21
4      2008-07-28
          ...    
819    2024-12-02
820    2024-12-09
821    2024-12-16
822    2024-12-23
823    2024-12-30
Name: DATE, Length: 824, dtype: object

In [30]:
data['Lingulodinium_polyedra']

time
0      446.0
1      446.0
2      446.0
3      446.0
4      446.0
       ...  
878      0.0
879      0.0
880      0.0
881      0.0
882    500.0
Name: Lingulodinium_polyedra, Length: 883, dtype: float64

In [116]:
def get_live_data(service_json_path):
    FILE_ID = "1bHrmzw5W5cz8DE-ggEwA3WSEDuszG72UUvKQRBBnLPA"     # your sheet
    SERVICE_JSON = service_json_path

    # --- auth ---
    creds = service_account.Credentials.from_service_account_file(
        SERVICE_JSON,
        scopes=["https://www.googleapis.com/auth/drive.readonly"],
    )
    drive = build("drive", "v3", credentials=creds)

    # --- export Google Sheet -> CSV ---
    request = drive.files().export_media(
        fileId=FILE_ID,
        mimeType="text/csv",
        # **required for Shared Drives** :contentReference[oaicite:0]{index=0}
    )

    fh = io.BytesIO()
    downloader = MediaIoBaseDownload(fh, request)
    done = False
    while not done:
        status, done = downloader.next_chunk()          # progress in chunks

    fh.seek(0)                     # rewind the buffer
    df = pd.read_csv(fh)           # straight into pandas
    return df
def adjust_live_data(past_data_path, target,json_key_path):  
    live_df = get_live_data(json_key_path)
    live_df.columns = (
        live_df.columns
        .str.replace(r"\s*\(.*?\)", "", regex=True)   # drop units in ( )
        .str.strip()                                 # remove leading/trailing spaces
        .str.replace(" ", "_")                       # spaces → underscores
    )

    # optional: collapse any accidental double-underscores
    live_df.columns = live_df.columns.str.replace(r"__+", "_", regex=True)
    live_df['time'] = live_df.index
    #Replace live data missing values with carter dataset
    paper_data = clean_data(past_data_path)
    # Convert the yyyymmdd integers to the same yyyy-mm-dd string format
    live_df["SampleID"] = (
        live_df["SampleID"]
            .astype(str)                       
            .str.zfill(8)                      
            .pipe(pd.to_datetime, format="%Y%m%d") 
            .dt.strftime("%Y-%m-%d")           # → "2025-04-28"
    )
    paper_data['DATE'] = pd.to_datetime(paper_data['DATE'])

    clean_dates = (
        live_df["SampleID"]
        .astype(str)                       # 2008-06-30 → "2008-06-30"
        .str.replace(r"\D", "", regex=True)  # drop non-digits → "20080630"
        .str.zfill(8)                      # make sure it's 8 chars
    )
    
    live_df["DATE"] = (
        pd.to_datetime(clean_dates, format="%Y%m%d")  # Timestamp dtype
        .dt.strftime("%Y-%m-%d")                      # "YYYY-MM-DD" strings
    )



    target_lookup = paper_data.set_index("DATE")[target]
    live_df[target] = live_df[target].fillna(live_df["DATE"].map(target_lookup))
    live_df = clean_data(live_df,path=False)
    return live_df
adjust_live_data(data_path,target,"/Users/athulithparaselli/Documents/GitHub/HAB-Prediction-Research/Code/APIKey/sacred-truck-461716-t5-2a6499bf0869.json")

/var/folders/5y/74w_zv855875xvjtmhp4g1xw0000gn/T/ipykernel_75917/2198128041.py:38: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  X_filled_forward = X.fillna(method='ffill').fillna(method='bfill')
/var/folders/5y/74w_zv855875xvjtmhp4g1xw0000gn/T/ipykernel_75917/2198128041.py:39: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  X_filled_backward = X.fillna(method='bfill').fillna(method='ffill')
/var/folders/5y/74w_zv855875xvjtmhp4g1xw0000gn/T/ipykernel_75917/2198128041.py:38: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  X_filled_forward = X.fillna(method='ffill').fillna(method='bfill')
/var/folders/5y/74w_zv855875xvjtmhp4g1xw0000gn/T/ipykernel_75917/2198128041.py:39: FutureWarning: DataFrame.fillna with 'method' is dep

,SampleID,Location_Code,Date_Collected,Time_Collected_PT,Temp,Chl_Volume_Filtered,Chl1,Chl2,Avg_Chloro,Phaeo1,...,Ceratium_spp.,Cochlodinium/Margalefidinium_spp.,Dinophysis_spp.,Gymnodinium_spp.,Lingulodinium_polyedra,Prorocentrum_spp.,Other_Dinoflagellates,Total_Phytoplankton,DATE,Time
time,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,19.8,100.0,1.81,1.83,1.82,0.59,...,1247.0,0.0,89.0,0.0,89.0,535.0,8822.0,20852.0,2008-06-30,0.0
1,NaN,NaN,NaN,NaN,17.3,100.0,2.68,2.73,2.71,1.24,...,1247.0,0.0,89.0,0.0,89.0,535.0,8822.0,20852.0,2008-07-07,1.0
2,NaN,NaN,NaN,NaN,21.7,100.0,0.99,1.00,0.99,0.55,...,1247.0,0.0,89.0,0.0,267.0,535.0,8822.0,20852.0,2008-07-14,2.0
3,NaN,NaN,NaN,NaN,20.2,100.0,1.21,1.24,1.23,0.49,...,1247.0,0.0,89.0,0.0,0.0,535.0,8822.0,20852.0,2008-07-21,3.0
4,NaN,NaN,NaN,NaN,22.4,100.0,2.37,2.44,2.41,0.41,...,1247.0,0.0,89.0,0.0,0.0,535.0,8822.0,20852.0,2008-07-28,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
878,NaN,NaN,NaN,NaN,16.9,100.0,1.21,1.19,1.20,0.49,...,626.0,0.0,0.0,0.0,0.0,8135.0,20026.0,124535.0,2025-04-28,878.0
879,NaN,NaN,NaN,NaN,16.8,100.0,1.21,1.19,1.20,0.49,...,626.0,0.0,0.0,0.0,0.0,8135.0,20026.0,124535.0,2025-05-05,879.0
880,NaN,NaN,NaN,NaN,17.2,100.0,1.21,1.19,1.20,0.49,...,626.0,0.0,0.0,0.0,0.0,8135.0,20026.0,124535.0,2025-05-12,880.0


In [68]:
df

,SampleID,Location Code,Date Collected (YYYY_MM_DD),Time Collected PT (hh:mm:ss),Temp (deg C),Chl Volume Filtered (mL),Chl1 (mg/m3),Chl2 (mg/m3),Avg Chloro (mg/m3),Phaeo1 (mg/m3),...,Akashiwo sanguinea (cells/L),Alexandrium spp. (cells/L),Ceratium spp.(cells/L),Cochlodinium/Margalefidinium spp. (cells/L),Dinophysis spp. (cells/L),Gymnodinium spp. (cells/L),Lingulodinium polyedra (cells/L),Prorocentrum spp. (cells/L),Other Dinoflagellates (cells/L),Total Phytoplankton (cells/L)
0,20080630,SIO,6/30/08,8:00:00,19.8,100,1.81,1.83,1.82,0.59,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20080707,SIO,7/7/08,8:00:00,17.3,100,2.68,2.73,2.71,1.24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20080714,SIO,7/14/08,7:54:00,21.7,100,0.99,1.00,0.99,0.55,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20080721,SIO,7/21/08,7:50:00,20.2,100,1.21,1.24,1.23,0.49,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20080728,SIO,7/28/08,9:44:00,22.4,100,2.37,2.44,2.41,0.41,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
878,20250428,SIO,4/28/25,12:21:00,16.9,100,1.21,1.19,1.20,0.49,...,626.0,0.0,626.0,0.0,0.0,0.0,0.0,8135.0,20026.0,124535.0
879,20250505,SIO,5/5/25,12:34:00,16.8,100,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN
880,20250512,SIO,5/12/25,NaN,NaN,100,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN
881,20250519,SIO,5/19/25,11:53:00,17.6,100,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN


In [121]:
pd.read_csv(data_path).columns

Index(['Unnamed: 0', 'Temp', 'Chl1', 'Chl2', 'Avg_Chloro', 'Phaeo1', 'Phaeo2',
       'Avg_Phaeo', 'Nitrate', 'Phosphate', 'Silicate', 'Nitrite', 'Ammonium',
       'CellCountDetection_Limit', 'Total_Diatoms', 'Akashiwo_sanguinea',
       'Total_Tripos', 'Total_Cochlodinium_spp', 'Lingulodinium_polyedra',
       'Total_Prorocentrum_spp', 'Total_Dinoflagellates',
       'Total_Phytoplankton', 'DATE', 'SURF_SAL_PSU', 'BOT_SAL_PSU',
       'SURF_TEMP_C', 'BOT_TEMP_C', 'WSPD', 'time', 'SURF_DENS_kgm3',
       'BOT_DENS_kgm3', 'AVG_TEMP_C', 'AVG_SAL_PSU', 'AVG_DENS_kgm3'],
      dtype='object')

In [122]:
df.columns

Index(['SampleID', 'Location Code', 'Date Collected (YYYY_MM_DD)',
       'Time Collected PT (hh:mm:ss)', 'Temp (deg C)',
       'Chl Volume Filtered (mL)', 'Chl1 (mg/m3)', 'Chl2 (mg/m3)',
       'Avg Chloro (mg/m3)', 'Phaeo1 (mg/m3)', 'Phaeo2 (mg/m3)',
       'Avg Phaeo (mg/m3)', 'Nitrate (uM)', 'Phosphate (uM)', 'Silicate (uM)',
       'Nitrite (uM)', 'Ammonium (uM)', 'DA Volume Filtered (mL)',
       'pDA (ng/mL)', 'Cell Count Detection Limit (cells/L)',
       'Pseudo-nitzschia delicatissima group (cells/L)',
       'Pseudo-nitzschia seriata group (cells/L)', 'Other Diatoms (cells/L)',
       'Akashiwo sanguinea  (cells/L)', 'Alexandrium spp. (cells/L)',
       'Ceratium spp.(cells/L)', 'Cochlodinium/Margalefidinium spp. (cells/L)',
       'Dinophysis spp. (cells/L)', 'Gymnodinium spp. (cells/L)',
       'Lingulodinium polyedra (cells/L)', 'Prorocentrum spp. (cells/L)',
       'Other Dinoflagellates (cells/L)', 'Total Phytoplankton (cells/L)'],
      dtype='object')

In [123]:
import pandas as pd
import numpy as np

# ------------------------------------------------------------------
# 0️⃣  Make sure every "(cells/L)" column is numeric
cells_cols = df.filter(regex=r"\(cells/L\)$").columns
df[cells_cols] = df[cells_cols].apply(pd.to_numeric, errors="coerce")

# ------------------------------------------------------------------
# 1️⃣  Tell Pandas which columns belong to each group
diatom_cols = [
    "Pseudo-nitzschia delicatissima group (cells/L)",
    "Pseudo-nitzschia seriata group (cells/L)",
    "Other Diatoms (cells/L)",
]

dinoflag_cols = [
    "Akashiwo sanguinea  (cells/L)",
    "Alexandrium spp. (cells/L)",
    "Ceratium spp.(cells/L)",
    "Cochlodinium/Margalefidinium spp. (cells/L)",
    "Dinophysis spp. (cells/L)",
    "Gymnodinium spp. (cells/L)",
    "Lingulodinium polyedra (cells/L)",
    "Prorocentrum spp. (cells/L)",
    "Other Dinoflagellates (cells/L)",
]

# ------------------------------------------------------------------
# 2️⃣  Add totals (skip NaNs but keep NaN if *every* entry is missing)
df["Total_Diatoms"]        = df[diatom_cols].sum(axis=1, min_count=1)
df["Total_Dinoflagellates"] = df[dinoflag_cols].sum(axis=1, min_count=1)


In [124]:
df

,SampleID,Location Code,Date Collected (YYYY_MM_DD),Time Collected PT (hh:mm:ss),Temp (deg C),Chl Volume Filtered (mL),Chl1 (mg/m3),Chl2 (mg/m3),Avg Chloro (mg/m3),Phaeo1 (mg/m3),...,Ceratium spp.(cells/L),Cochlodinium/Margalefidinium spp. (cells/L),Dinophysis spp. (cells/L),Gymnodinium spp. (cells/L),Lingulodinium polyedra (cells/L),Prorocentrum spp. (cells/L),Other Dinoflagellates (cells/L),Total Phytoplankton (cells/L),Total_Diatoms_cells_L,Total_Dinoflagellates_cells_L
0,20080630,SIO,6/30/08,8:00:00,19.8,100,1.81,1.83,1.82,0.59,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20080707,SIO,7/7/08,8:00:00,17.3,100,2.68,2.73,2.71,1.24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20080714,SIO,7/14/08,7:54:00,21.7,100,0.99,1.00,0.99,0.55,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20080721,SIO,7/21/08,7:50:00,20.2,100,1.21,1.24,1.23,0.49,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20080728,SIO,7/28/08,9:44:00,22.4,100,2.37,2.44,2.41,0.41,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
878,20250428,SIO,4/28/25,12:21:00,16.9,100,1.21,1.19,1.20,0.49,...,626.0,0.0,0.0,0.0,0.0,8135.0,20026.0,124535.0,95122.0,29413.0
879,20250505,SIO,5/5/25,12:34:00,16.8,100,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0
880,20250512,SIO,5/12/25,NaN,NaN,100,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0
881,20250519,SIO,5/19/25,11:53:00,17.6,100,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0
